In [1]:
import json
import math
import numpy as np
import random
import scipy.stats

In [2]:
from utils import join_reference_set_and_results, compute_sign_rate_found, create_map

In [3]:
with open('observational_results.txt') as f:
    observational_results = [json.loads(a) for a in f]

reference_set_names = ['../raw_reference_set.txt', '../reference_set.txt', '../multi_rct_reference_set.txt']
reference_sets = []

for reference_set_name in reference_set_names:
    with open(reference_set_name) as f:
        reference_sets.append([json.loads(a) for a in f])

In [4]:
filtered_maps = []
for reference_set in reference_sets:
    joined = join_reference_set_and_results(reference_set, observational_results)
    filtered_maps.append(create_map([a for a in joined if 'postmean' in a]))

In [5]:
total_keys = set()
for filtered_map in filtered_maps:
    total_keys |= set(filtered_map.keys())

total_keys = sorted(list(total_keys))

In [6]:
random.seed(123154)

performances = [[] for _ in range(len(filtered_maps))]

method_type = 'cox'
method_key = 'logistic_match'

lower_limit = 1.5
upper_limit = 4

def get_performance_at(sample, filtered_map, value):
    filtered = [filtered_map.get(k) for k in sample if k in filtered_map]
    sorted_by_bayes = sorted(filtered, key=lambda v: v['postmean'])
    
    postmean, fraction_correct, fraction_found, num_significant = compute_sign_rate_found(sorted_by_bayes, method_type, method_key)

    return np.interp(-value, -postmean, fraction_correct), np.interp(-value, -postmean, fraction_found)


def get_average_performance_between(sample, filtered_map, lower_limit, upper_limit):
    filtered = [filtered_map.get(k) for k in sample if k in filtered_map]
    sorted_by_bayes = sorted(filtered, key=lambda v: v['postmean'])
    
    postmean, fraction_correct, fraction_found, num_significant = compute_sign_rate_found(sorted_by_bayes, method_type, method_key)

    first_less_than_limit = np.argmax(postmean < upper_limit)
    assert postmean[first_less_than_limit] < upper_limit
    assert postmean[first_less_than_limit - 1] >= upper_limit
    
    postmean[first_less_than_limit - 1] = upper_limit
    
    postmean = postmean[first_less_than_limit - 1:]
    fraction_correct = fraction_correct[first_less_than_limit - 1:]
    fraction_found = fraction_found[first_less_than_limit - 1:]

    first_greater_than = np.argmax(postmean <= lower_limit)
    assert postmean[first_greater_than] <= lower_limit
    assert postmean[first_greater_than - 1] > lower_limit
    
    postmean[first_greater_than - 1] = lower_limit
    
    postmean = postmean[:first_greater_than]
    fraction_correct = fraction_correct[:first_greater_than]
    fraction_found = fraction_found[:first_greater_than]

    return -np.trapz(fraction_correct, postmean) / (upper_limit - lower_limit), -np.trapz(fraction_found, postmean) / (upper_limit - lower_limit)

print(lower_limit, upper_limit)
for reference_set_name, filtered_map in zip(reference_set_names, filtered_maps):
    print(reference_set_name, get_performance_at(total_keys, filtered_map, 3))

for i in range(1000):
    if i % 100 == 0:
        print(i)
    sample = random.choices(total_keys, k=len(total_keys))
    for performance, filtered_map in zip(performances, filtered_maps):
        performance.append(get_performance_at(sample, filtered_map, 3))

1.5 4
../raw_reference_set.txt (0.9011411272247049, 0.451861421602523)
../reference_set.txt (1.0, 0.57645288354651)
../multi_rct_reference_set.txt (0.8716904686889657, 0.6128068711938315)
0
100
200
300
400
500
600
700
800
900


In [10]:
for i in range(1, len(reference_set_names)):
    print(reference_set_names[0], reference_set_names[i])
    for index, name in enumerate(["concordance", "recovery"]):
        helper = lambda a: a[index]
        print(name)

        for other in (0, i):
            print(reference_set_names[other], f'{100 * helper(get_performance_at(total_keys, filtered_maps[other], 3)):0.2f}')
        
        deltas = []
        for a, b in zip(performances[0], performances[i]):
            deltas.append(helper(a) - helper(b))
    
        z = np.mean(deltas) / np.std(deltas, ddof=1)
        p = scipy.stats.norm.sf(abs(z)) * 2
        print(z, p)
    
        print(np.quantile(deltas, [0.025, 0.975]))
        
        print(np.mean(deltas), np.std(deltas,ddof=1))

../raw_reference_set.txt ../reference_set.txt
concordance
../raw_reference_set.txt 90.11
../reference_set.txt 100.00
-1.894826532945524 0.05811538681705672
[-0.18533475  0.00692144]
-0.09159909291047107 0.04834167736087139
recovery
../raw_reference_set.txt 45.19
../reference_set.txt 57.65
-1.2879981228791768 0.19774661661013038
[-0.28677459  0.04997231]
-0.11002904356235951 0.08542640055748048
../raw_reference_set.txt ../multi_rct_reference_set.txt
concordance
../raw_reference_set.txt 90.11
../multi_rct_reference_set.txt 87.17
0.4896945790355245 0.6243500384352225
[-0.08592109  0.1980854 ]
0.03526094337318743 0.07200599084154749
recovery
../raw_reference_set.txt 45.19
../multi_rct_reference_set.txt 61.28
-1.6354482012788687 0.10195512491844887
[-0.34301118  0.03062251]
-0.1524082267361528 0.09319049457939078
